In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('Crash_Reporting_-_Drivers_Data.csv', low_memory=False)
cleaned_data = data

In [3]:
# Vehicle Make cleaning
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace(['TOYT','TOYO','TOYTA', 'TOY'], 'TOYOTA')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace('HOND', 'HONDA')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace(['CHEVY', 'CHEV','CHEYV'], 'CHEVROLET')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace('NISS', 'NISSAN')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace(['HYUN','HYUNDIA','HYUNUNDAI','HYUNUDAI','HYUUNDAI'], 'HYUNDAI')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace(['MERCEDES-BE','MERCEDESS','MERCEDES ZBENZ','MERCEDES ENZ','MERCEDEZ BENS','MERCEDS','MERDECES','MERCRDEZ BENZ','MERCEZ','MERCENDES','MERCEES','MERCEEDEZ','MERCEES','MERZ','MERCEDES', 'MERC','MERCEDES BENZ','MERCEDEZ','MEZ','MERZEDEZ','MERZEDES BENZ','MERZ 4S','MERX','MERS-BENZ','MERECEDEZ','MERDEDES','MERDECES-BENZ','MERCRY','MERDECES-BENZ'], 'MERCEDES-BENZ')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace(['VOLK','VOLKS','VW'], 'VOLKSWAGEN')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace('ACUR', 'ACURA')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace('SUBA', 'SUBARU')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace('DODG', 'DODGE')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace(['THOM', 'THOMAS','THMS'], 'THOMAS BUILT')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace('MAZD', 'MAZDA')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace(['LEXS','LEXU','LESUX','LEXUUS','LEXUSZ','LEXUSS','LEXI'], 'LEXUS')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace('FRHT', 'FREIGHTLINER')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace('VOLV', 'VOLVO')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace('INFI', 'INFINITI')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace(['GILL','GILG','GILLLIG BU','GILLS','GILLMAN'],'GILLIG')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace('CHRY', 'CHRYSLER')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace('BUIC', 'BUICK')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace(['CADI','CADDILAC','CADILACC','CADILLA'], 'CADILLAC')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace(['INFINITY','INFINITIG'], 'INFINITI')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace('LINC', 'LINCOLN')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace('INTL', 'INTERNATIONAL')
cleaned_data['Vehicle Make'] = cleaned_data['Vehicle Make'].replace('NISSIAN', 'NISSAN')

#Substance cleaning
cleaned_data['Driver Substance Abuse'] = cleaned_data['Driver Substance Abuse'].str.upper()
cleaned_data['Driver Substance Abuse'] = cleaned_data['Driver Substance Abuse'].replace('UNKNOWN, UNKNOWN', 'UNKNOWN')
cleaned_data['Driver Substance Abuse'] = cleaned_data['Driver Substance Abuse'].replace('SUSPECT OF ALCOHOL USE, UNKNOWN', 'SUSPECT OF ALCOHOL USE, NOT SUSPECT OF DRUG USE')

#Traffic controls
cleaned_data['Traffic Control'] = cleaned_data['Traffic Control'].replace('TRAFFIC CONTROL SIGNAL', 'TRAFFIC SIGNAL')
cleaned_data['Traffic Control'] = cleaned_data['Traffic Control'].replace('FLASHING TRAFFIC CONTROL SIGNAL','FLASHING TRAFFIC SIGNAL')
cleaned_data['Traffic Control'] = cleaned_data['Traffic Control'].fillna('UNKNOWN')

# Weather
cleaned_data['Weather'] = cleaned_data['Weather'].str.upper()
cleaned_data['Weather'] = cleaned_data['Weather'].fillna('UNKNOWN')


#Surface condition
cleaned_data['Surface Condition'] = cleaned_data['Surface Condition'].str.upper()
cleaned_data['Surface Condition'] = cleaned_data['Surface Condition'].replace(['WATER(STANDING/MOVING)','WATER (STANDING, MOVING)'],'WET')
cleaned_data['Surface Condition'] = cleaned_data['Surface Condition'].replace('ICE/FROST','ICE')


#Light
cleaned_data['Light'] = cleaned_data['Light'].str.upper()
cleaned_data['Light'] = cleaned_data['Light'].replace('DARK - LIGHTED','DARK LIGHTS')
cleaned_data['Light'] = cleaned_data['Light'].replace('DARK - NOT LIGHTED','DARK NO LIGHTS')
cleaned_data['Light'] = cleaned_data['Light'].replace(['DARK -- UNKNOWN LIGHTING','DARK - UNKNOWN LIGHTING'],'DARK')


In [4]:
#Fixing Differences in Upper and Lower case strings
cleaned_data['Vehicle Damage Extent'] = cleaned_data['Vehicle Damage Extent'].str.upper()
cleaned_data['Injury Severity'] = cleaned_data['Injury Severity'].str.upper()
cleaned_data['Traffic Control'] = cleaned_data['Traffic Control'].str.upper()
cleaned_data['Surface Condition'] = cleaned_data['Surface Condition'].str.upper()

In [5]:
#Creating the subset for label propegation
columns = ['Speed Limit','Injury Severity','Collision Type','Weather','Circumstance','Surface Condition','Light','Traffic Control','Vehicle First Impact Location','Vehicle Damage Extent']
subset = cleaned_data[columns]
subset = subset.dropna()
#adding a label for the final label propegation data
large_data = cleaned_data[cleaned_data['Circumstance'].isna()]
large_data = large_data[columns]
large_data = large_data.drop('Circumstance', axis=1)
large_data['Surface Condition'].fillna('UNKNOWN')
large_data['Predicted_Label'] = -1

In [19]:
subset['Circumstance'].value_counts().head(15)

Circumstance
RAIN, SNOW, WET                                   11590
N/A, WET                                           8480
N/A, RAIN, SNOW                                    1043
SLEET, HAIL, FREEZ. RAIN, WET                      1001
N/A, VISION OBSTRUCTION (INCL. BLINDED BY SUN)      962
Failed to Yield Right-of-Way                        949
ICY OR SNOW-COVERED, RAIN, SNOW                     887
ANIMAL, N/A                                         676
BACKUP DUE TO REGULAR CONGESTION, N/A               648
ICY OR SNOW-COVERED, N/A                            621
N/A, ROAD UNDER CONSTRUCTION/MAINTENANCE            456
Followed Too Closely                                426
Other Improper Action                               416
DEBRIS OR OBSTRUCTION, N/A                          292
ICY OR SNOW-COVERED, SLEET, HAIL, FREEZ. RAIN       253
Name: count, dtype: int64

In [20]:
# Define keywords for collisions (1) and non-collisions (0)
collision_keywords = ['Other Improper Action','Followed Too Closely','Followed Too Closely ','Improper Passing','Too Fast For Conditions','Ran Red Light','Failed to Keep in Proper Lane','Failed to Yield Right-of-Way','Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner', 'Improper Turn']
non_collision_keywords = ['ANIMAL','VISION OBSTRUCTION (INCL. BLINDED BY SUN)','ICY OR SNOW-COVERED, RAIN, SNOW', 'BACKUP DUE TO REGULAR CONGESTION', 'DEBRIS OR OBSTRUCTION']

# Assign labels:

subset['Collision'] = -1  # Default all to -1
subset.loc[subset['Circumstance'].str.contains('|'.join(collision_keywords), case=False, na=False), 'Collision'] = 1
subset.loc[subset['Circumstance'].str.contains('|'.join(non_collision_keywords), case=False, na=False), 'Collision'] = 0

/tmp/ipykernel_6810/1322676498.py:9: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  subset.loc[subset['Circumstance'].str.contains('|'.join(non_collision_keywords), case=False, na=False), 'Collision'] = 0


In [21]:
subset['Collision'].value_counts()

Collision
-1    26694
 1     3950
 0     3050
Name: count, dtype: int64

In [22]:
import numpy as np
from sklearn.semi_supervised import LabelPropagation
from sklearn.preprocessing import OneHotEncoder 

In [23]:
print(np.__version__)

2.2.2


In [24]:
# Define categorical features
categorical_cols = ['Surface Condition', 'Weather', 'Traffic Control', 'Speed Limit']

# One-Hot Encode categorical features for the subset
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
X_categorical_subset = encoder.fit_transform(subset[categorical_cols])

# Prepare labels for Label Propagation
y_subset = subset['Collision']

# Train Label Propagation Model
label_prop_model = LabelPropagation(kernel='knn')
label_prop_model.fit(X_categorical_subset, y_subset)

# Transform the large dataset
X_categorical_large = encoder.transform(large_data[categorical_cols])

# Predict labels
subset['Predicted_Label'] = label_prop_model.predict(X_categorical_subset)
large_data['Predicted_Label'] = label_prop_model.predict(X_categorical_large)

# Output results
print("Predicted Labels in Subset:")
print(subset['Predicted_Label'].value_counts())

print("\nPredicted Labels in Large Dataset:")
print(large_data['Predicted_Label'].value_counts())



/home/jianderi/anaconda3/lib/python3.11/site-packages/sklearn/semi_supervised/_label_propagation.py:231: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer


Predicted Labels in Subset:
Predicted_Label
0    27153
1     6541
Name: count, dtype: int64

Predicted Labels in Large Dataset:
Predicted_Label
0    90345
1    64515
Name: count, dtype: int64


In [25]:
# Ensure both datasets have the same columns
subset['Predicted_Label'] = subset['Predicted_Label'].astype(int)  # Ensure dtype consistency
large_data['Predicted_Label'] = large_data['Predicted_Label'].astype(int)  

# Combine into a master dataset
master_data = pd.concat([subset, large_data], ignore_index=True)
master_data.drop(['Collision','Circumstance'], axis=1, inplace=True)
# Check the shape and column consistency
print(master_data['Predicted_Label'].value_counts().head())

Predicted_Label
0    117498
1     71056
Name: count, dtype: int64


In [26]:
master_data.to_csv('Labeled_data.csv', index=False)